In [1]:
from dask.distributed import Client
from dask_kubernetes.operator import KubeCluster
from src.cloud_manager import VMManager

In [3]:
cluster: KubeCluster = KubeCluster(name="mycluster",
                                   image='hermelinkluntjes/thesis:test',
                                   n_workers=0)
client = Client(cluster)
client

/opt/conda/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/opt/conda/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Task exception was never retrieved
future: <Task finished name='Task-35' coro=<PortForward._sync_sockets() done, defined at 
/opt/conda/lib/python3.10/site-packages/kr8s/_portforward.py:167> exception=ExceptionGroup('unhandled errors in a 
TaskGroup', [ConnectionClosedError('TCP socket closed')])>
  + Exception Group Traceback (most recent call last):
  |   File "/opt/conda/lib/python3.10/site-packages/kr8s/_portforward.py", line 171, in _sync_sockets
  |     async with anyio.create_task_group() as tg:
  |   File "/opt/conda/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 664, in __aexit__
  |     raise BaseExceptionGroup(
  | exceptiongroup.ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "/opt/conda/lib/python3.10/site-packages/kr8s/_portforward.py", line 183, in _tcp_to_ws
    |     raise ConnectionClosedError("TCP socket closed")
    | kr8s._exceptions.ConnectionClosedError: TCP socket closed
    +------------------------------------

/opt/conda/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Connection method: Cluster object,Cluster type: dask_kubernetes.KubeCluster
Dashboard: http://localhost:55410/status,
Dashboard: http://localhost:55410/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.0.17:8786,Workers: 0
Dashboard: http://10.12.0.17:8787/status,Total threads: 0
Started: 1 minute ago,Total memory: 0 B


In [ ]:
worker_group_config = {
   "name": "highmem",
   "n_workers": 2,
   "resources":{
      "requests": 
         {"memory": "6Gi",
          "cpu": "3"}
      }
}
cluster.add_worker_group(**worker_group_config)

In [ ]:
cluster.scale(2, worker_group="highmem")
# Dont scale down while cluster scales up -> makes for errors
client.wait_for_workers(2)

In [ ]:
with VMManager(cluster) as vmmanager:
    print(vmmanager.cluster)